# GapsTagger

In [1]:
from estnltk import Text
from estnltk.layer import Layer
from estnltk.spans import Span
from estnltk.taggers.gaps_tagging.gaps_tagger import GapsTagger

In [2]:
text = Text('Üks kaks kolm neli viis kuus seitse.')
layer_1 = Layer('test_1')
layer_1.add_span(Span(4, 8))
layer_1.add_span(Span(9, 13))
layer_1.add_span(Span(24, 28))
text['test_1'] = layer_1

layer_2 = Layer('test_2')
layer_2.add_span(Span(4, 8))
layer_2.add_span(Span(9, 18))
layer_2.add_span(Span(35, 36))
text['test_2'] = layer_2

## Example 1

In [3]:
gaps_tagger = GapsTagger('simple_gaps', ['test_1', 'test_2'])
gaps_tagger.tag(text)
text.simple_gaps

SL[Span(Üks , {}),
Span( , {}),
Span( viis , {}),
Span( seitse, {})]

The following illustrates examples 1 and 2.

    text:           'Üks kaks kolm neli viis kuus seitse.'
    test_1:             'kaks'kolm'         'kuus'      
    test_2:             'kaks'kolm neli'               '.'
    simple_gaps:    'Üks '  ' '       ' viis '  ' seitse'
    gaps:           'Üks'              'viis'    'seitse'

## Example 2

In [4]:
def trim(t:str) -> str:
    return t.strip()

def decorator(text:str):
    return {'gap_length':len(text)}

gaps_tagger = GapsTagger(layer_name='gaps',
                         input_layers=['test_1', 'test_2'],
                         trim=trim,
                         decorator=decorator,
                         attributes=['gap_length'])
gaps_tagger

GapsTagger(decorator function=<function decorator at 0x7fea2e6027b8>, trim function=<function trim at 0x7fea2e6029d8>)

In [5]:
gaps_tagger.tag(text)
text.gaps

SL[Span(Üks, {'gap_length': 3}),
Span(viis, {'gap_length': 4}),
Span(seitse, {'gap_length': 6})]

## Example 3: enveloping layers

The gaps layer of enveloping layers is an unambiguous enveloping layer. All input layers must be enveloping the same layer. Input layers can be ambiguous or unambiguous.

In [6]:
from estnltk.spans import SpanList

text = Text('Üks kaks kolm neli viis kuus seitse.')
text.tag_layer(['words'])

layer = Layer('test_3', enveloping='words')

spl = SpanList()
spl.spans = text.words[0:2]
layer.add_span(spl)

spl = SpanList()
spl.spans = text.words[3:4]
layer.add_span(spl)

text['test_3'] = layer
text.test_3

SL[SL[Span(Üks, {'normalized_form': None}),
Span(kaks, {'normalized_form': None})],
SL[Span(neli, {'normalized_form': None})]]

In [7]:
layer = Layer('test_4', enveloping='words', ambiguous=True)

spl = SpanList()
spl.spans = text.words[3:5]
layer.add_span(spl)

spl = SpanList()
spl.spans = text.words[3:5]
layer.add_span(spl)

text['test_4'] = layer
text.test_4.spans

SL[SL[SL[Span(neli, {'normalized_form': None}),
Span(viis, {'normalized_form': None})],
SL[Span(neli, {'normalized_form': None}),
Span(viis, {'normalized_form': None})]]]

In [8]:
def decorator(spans):
    return {'gap_word_count': len(spans)}

gaps_tagger = GapsTagger(layer_name='gaps',
                         input_layers=['test_3', 'test_4'],
                         decorator=decorator,
                         attributes=['gap_word_count'])
gaps_tagger

GapsTagger(decorator function=<function decorator at 0x7fea2e602620>, trim function=<function trim at 0x7fea2e602510>)

In [9]:
gaps_tagger.tag(text)
text.gaps

SL[SL[Span(kolm, {'normalized_form': None})],
SL[Span(kuus, {'normalized_form': None}),
Span(seitse, {'normalized_form': None}),
Span(., {'normalized_form': None})]]